In [62]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import scipy as sp
path='/home/student303/zz/almama/'

In [63]:
train = pd.read_hdf(path+ 'somefeature/two_column_train_cnt.h5', key = 'a')
test = pd.read_hdf(path+ 'somefeature/two_column_test_cnt.h5', key = 'a')
#y_train = pd.read_csv(path + 'somefeature/y_train.csv')

In [38]:
train.head()

context_id  context_page_id  context_timestamp  day  hour  \
0  5035994110420503338             4002         1537465939   21     1   
1  7972645707776427108             4015         1537537076   21    21   
2  6797298244965748241             4002         1537539269   21    22   
3     7455406367907195             4001         1537509234   21    13   
4  1422823996606455864             4001         1537494715   21     9   

           instance_id  is_trade        item_brand_id  item_category_hash  \
0  4293715837363757697       0.0  1975590437749032870                5332   
1    70764105903083828       0.0  9057103201734987852                9521   
2  3954908518456917085       0.0  5520678735822176314                1443   
3  4521314384240415059       0.0  9057103201734987852                9521   
4  6005676298575614673       0.0  5520678735822176314                1443   

                        item_category_list  \
0  7908382889764677758;5799347067982556520   
1  7908382889764677758;8277336076276184272   
2  7908382889764677758;5755694407684602296   
3  7908382889764677758;8277336076276184272   
4  7908382889764677758;5755694407684602296   

                    ...                     \
0                   ...                      
1                   ...                      
2                   ...                      
3                   ...                      
4                   ...                      

   user_age_level->user_gender_id_cnt  user_age_level->user_id_cnt  \
0                                1021                          NaN   
1                               62287                          NaN   
2                               62287                          NaN   
3                               62287                          NaN   
4                                1722                          NaN   

   user_age_level->user_occupation_id_cnt  \
0                                    4402   
1                                   48768   
2                                   48768   
3                                   48768   
4                                    1697   

   user_age_level->user_star_level_cnt user_gender_id->user_id_cnt  \
0                                605.0                         NaN   
1                              12037.0                         NaN   
2                              12037.0                         NaN   
3                              12094.0                         NaN   
4                                433.0                         NaN   

   user_gender_id->user_occupation_id_cnt  \
0                                  104100   
1                                  104100   
2                                  104100   
3                                  104100   
4                                  104100   

   user_gender_id->user_star_level_cnt  user_id->user_occupation_id_cnt  \
0                                26262                              NaN   
1                                26262                              NaN   
2                                26262                              NaN   
3                                23284                              NaN   
4                                21796                              NaN   

  user_id->user_star_level_cnt  user_occupation_id->user_star_level_cnt  
0                          NaN                                  24198.0  
1                          NaN                                  24198.0  
2                          NaN                                  24198.0  
3                          NaN                                  17823.0  
4                          NaN                                  22041.0  

[5 rows x 263 columns]

In [64]:
droplist = ['item_category_list',
            'item_property_list',
            'user_id',
            'context_id',
           #'context_page_id',
            'predict_category_property',
            #'shop_id'
           ]
df_test = test.drop(droplist, axis=1)
df_train = train.drop(droplist, axis=1)
# df_train.drop('is_trade', axis=1, inplace = True)
combine = [df_train, df_test]

In [65]:
df_train.columns

Index(['context_page_id', 'context_timestamp', 'day', 'hour', 'instance_id',
       'is_trade', 'item_brand_id', 'item_category_hash', 'item_city_id',
       'item_collected_level',
       ...
       'user_age_level->user_gender_id_cnt', 'user_age_level->user_id_cnt',
       'user_age_level->user_occupation_id_cnt',
       'user_age_level->user_star_level_cnt', 'user_gender_id->user_id_cnt',
       'user_gender_id->user_occupation_id_cnt',
       'user_gender_id->user_star_level_cnt',
       'user_id->user_occupation_id_cnt', 'user_id->user_star_level_cnt',
       'user_occupation_id->user_star_level_cnt'],
      dtype='object', length=258)

In [66]:
def logloss(act, pred):
    epsilon = 1e-15
    pred = sp.maximum(epsilon, pred)
    pred = sp.minimum(1-epsilon, pred)
    ll = sum(act*sp.log(pred) + sp.subtract(1,act)*sp.log(sp.subtract(1,pred)))
    ll = ll * -1.0/len(act)
    return ll

## train LGBM model

In [67]:
train_train = df_train[df_train.day != 24]
train_valid = df_train[df_train.day == 24]

train_X = train_train.drop('is_trade', axis=1)
train_y = train_train['is_trade'].values

valid_X = train_valid.drop('is_trade', axis=1)
valid_y = train_valid['is_trade'].values

d_train = lgb.Dataset(train_X, label=train_y)
d_valid = lgb.Dataset(valid_X, label=valid_y)
watchlist = [d_train, d_valid]

In [68]:
print('LGBM training')
#lgb_train = lgb.Dataset(df_train.values, label = y_train.is_trade)
params = {
    'task': 'train',
    #'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'max_depth':5,
#    'num_leaves': 6,
#    'min_data_in_leaf':1,
    #'max_bin':150,  
    'learning_rate': 0.1,
    'feature_fraction': 0.5,
    'bagging_fraction':0.8,
    #'bagging_freq': 5,
    'verbose': 0,
}
model = lgb.train(params, train_set=d_train, num_boost_round=6000, valid_sets=watchlist, \
                  early_stopping_rounds=500, verbose_eval=50)
print('LGBM training done')

LGBM training
Train until valid scores didn't improve in 500 rounds.
[50]	training's binary_logloss: 0.0848769	valid_1's binary_logloss: 0.0828616
[100]	training's binary_logloss: 0.0779862	valid_1's binary_logloss: 0.0821229
[150]	training's binary_logloss: 0.073336	valid_1's binary_logloss: 0.0826514
[200]	training's binary_logloss: 0.0689325	valid_1's binary_logloss: 0.0833475
[250]	training's binary_logloss: 0.0651541	valid_1's binary_logloss: 0.0835234
[300]	training's binary_logloss: 0.0614281	valid_1's binary_logloss: 0.0838962
[350]	training's binary_logloss: 0.0579448	valid_1's binary_logloss: 0.0841337
[400]	training's binary_logloss: 0.0546857	valid_1's binary_logloss: 0.084582
[450]	training's binary_logloss: 0.0515197	valid_1's binary_logloss: 0.0850613
[500]	training's binary_logloss: 0.0485329	valid_1's binary_logloss: 0.0853868
[550]	training's binary_logloss: 0.0456278	valid_1's binary_logloss: 0.0855637
Early stopping, best iteration is:
[84]	training's binary_logloss

## Add

In [71]:
droplist = ['item_category_list',
            'item_property_list',
            'user_id',
            'context_id',
           #'context_page_id',
            'predict_category_property',
            #'shop_id'
           ]
df_test = test.drop(droplist + newdropfeature, axis=1)
df_train = train.drop(droplist + newdropfeature, axis=1)
# df_train.drop('is_trade', axis=1, inplace = True)
combine = [df_train, df_test]

train_X = train_train.drop(newdropfeature+['is_trade'], axis=1)
train_y = train_train['is_trade'].values

valid_X = train_valid.drop(newdropfeature+['is_trade'], axis=1)
valid_y = train_valid['is_trade'].values

d_train = lgb.Dataset(train_X, label=train_y)
d_valid = lgb.Dataset(valid_X, label=valid_y)
watchlist = [d_train, d_valid]

print('LGBM training')
#lgb_train = lgb.Dataset(df_train.values, label = y_train.is_trade)
params = {
    'task': 'train',
    #'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'max_depth':5,
#    'num_leaves': 6,
#    'min_data_in_leaf':1,
    #'max_bin':150,  
    'learning_rate': 0.1,
    'feature_fraction': 0.8,
    'bagging_fraction':0.8,
    #'bagging_freq': 5,
    'verbose': 0,
}
model = lgb.train(params, train_set=d_train, num_boost_round=6000, valid_sets=watchlist, \
                  early_stopping_rounds=50, verbose_eval=10)
print('LGBM training done')

LGBM training
Train until valid scores didn't improve in 50 rounds.
[10]	training's binary_logloss: 0.237552	valid_1's binary_logloss: 0.235928
[20]	training's binary_logloss: 0.127379	valid_1's binary_logloss: 0.124597
[30]	training's binary_logloss: 0.0964447	valid_1's binary_logloss: 0.0930869
[40]	training's binary_logloss: 0.0878466	valid_1's binary_logloss: 0.0848981
[50]	training's binary_logloss: 0.0847594	valid_1's binary_logloss: 0.0828249
[60]	training's binary_logloss: 0.0829025	valid_1's binary_logloss: 0.0822577
[70]	training's binary_logloss: 0.0812724	valid_1's binary_logloss: 0.0821528
[80]	training's binary_logloss: 0.0799706	valid_1's binary_logloss: 0.0821338
[90]	training's binary_logloss: 0.0788498	valid_1's binary_logloss: 0.0821338
[100]	training's binary_logloss: 0.0778841	valid_1's binary_logloss: 0.0822353
[110]	training's binary_logloss: 0.0768472	valid_1's binary_logloss: 0.0822202
Early stopping, best iteration is:
[67]	training's binary_logloss: 0.0816566

In [70]:
newdropfeature=[]
for i in range(len(train_X.columns)):
    if model.feature_importance()[i]<=1:
        newdropfeature.append(train_X.columns[i])
    if model.feature_importance()[i]>20:
        print(train_X.columns[i])

context_timestamp
hour
item_price_level
item_sales_level
shop_score_delivery
shop_score_service
user_age_level
hour->item_id_cnt
hour->user_gender_id_cnt
item_brand_id->item_pv_level_cnt
item_category_hash->min_cnt
item_collected_level->item_sales_level_cnt


In [58]:
newdropfeature.remove('day')

## predict test prob 

In [107]:
instance_id=df_test.instance_id.values 
prob = model.predict(df_test, num_iteration=model.best_iteration )

output=pd.DataFrame({'instance_id':instance_id,'predicted_score':prob})  
output['predicted_score'] = output['predicted_score'].apply(lambda x:round(x,10))
print(output.head())
#output.to_hdf(path + 'output/submission.h5',index=False)
#print('the output path is '+path)

           instance_id  predicted_score
0  2475218615076601065          0.00791
1   398316874173557226          0.01633
2  6586402638209028583          0.01563
3  1040996105851528465          0.01780
4  6316278569655873454          0.01189
